In [6]:
import refinitiv.dataplatform as rdp
session = rdp.DesktopSession(app_key='<App Key>')
session.open()
endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics-curves-and-surfaces/v1/surfaces")
print("session opened and end-point set")

session opened and end-point set


In [7]:
import requests
import json
import pandas as pd
import numpy as np

In [8]:
request_body={
    "universe": [
        {
            "surfaceTag": "USD_Swaption_Vola",
            "underlyingType": "Swaption",
            "underlyingDefinition": {
                "instrumentCode": "USD",
            },
            "surfaceParameters": {
                #"xAxis": "Strike",
                #"yAxis": "Tenor",
                "calculationDate":"2022-03-31",
                #"outputVolatilityType": "LogNormalVolatility"
            },
            "surfaceLayout": {
                #"format": "Matrix",
                #"zValues": Tenors,
                #"yValues": ["1Y", "2Y", "3Y"],
            }
        }],
}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)

In [9]:
frames = []
surfdict = response.data.raw['data'][0]['surface']
for key in surfdict:
    surfacedata = np.array(surfdict[key])
    df = pd.DataFrame(data=surfacedata[1:,1:], index=surfacedata[1:,0], columns=surfacedata[0,1:])
    frames.append(df)

surface = pd.concat(frames, keys=surfdict.keys())
surface

-2.00   -1.50   -1.00   -0.50   -0.25    0.00    0.25    0.50  \
1M  1Y     0.00  105.54  108.87  110.68  111.87  113.50  115.65  118.35   
    2Y   111.03  114.79  116.67  118.93  120.55  122.60  125.10  128.03   
    3Y   111.20  114.49  116.28  118.57  120.22  122.30  124.82  127.78   
    4Y   109.84  113.79  115.69  117.95  119.56  121.60  124.09  127.03   
    5Y   107.76  112.69  114.73  116.94  118.51  120.50  122.95  125.86   
...         ...     ...     ...     ...     ...     ...     ...     ...   
30Y 10Y    0.00    0.00   35.56   53.82   54.01   53.90   54.96   57.90   
    15Y    0.00    0.00   12.97   52.23   51.92   51.90   53.86   58.15   
    20Y    0.00    0.00    0.00   44.13   48.09   50.50   53.23   56.98   
    25Y    0.00    0.00   18.18   50.17   50.01   49.90   51.43   55.10   
    30Y    0.00    0.00   31.95   49.60   49.81   49.70   50.60   53.19   

           1.00    1.50    2.00  
1M  1Y   125.29  133.86  143.48  
    2Y   135.10  143.42  152.58  
    3Y   134.86  143.16  152.29  
    4Y   134.09  142.42  151.58  
    5Y   132.91  141.26  150.48  
...         ...     ...     ...  
30Y 10Y   68.09   80.73   94.04  
    15Y   70.67   84.95   99.53  
    20Y   67.20   79.22   91.81  
    25Y   66.40   79.58   93.15  
    30Y   62.30   73.72   85.79  

[224 rows x 11 columns]

In [12]:
surfaceATM = surface['0.00'].astype(float).div(10000)
surfaceATM = surfaceATM.reset_index(level=[0,1])
surfaceATM.set_axis(['ExpiryTenor', 'SwapTenor', 'QuotedVol'], axis=1, inplace=True)
surfaceATM["Strike"] = ""
surfaceATM["StrikeQuotationType"] = "ATM"
surfaceATM.insert(loc=0, column='Quotationmodel', value="Bachelier")
surfaceATM.insert(loc=0, column='AsOfDate', value="31.03.2022")
surfaceATM.insert(loc=0, column='Currency', value="USD")
surfaceATM

,Currency,AsOfDate,Quotationmodel,ExpiryTenor,SwapTenor,QuotedVol,Strike,StrikeQuotationType
0,USD,31.03.2022,Bachelier,1M,1Y,0.01135,,ATM
1,USD,31.03.2022,Bachelier,1M,2Y,0.01226,,ATM
2,USD,31.03.2022,Bachelier,1M,3Y,0.01223,,ATM
3,USD,31.03.2022,Bachelier,1M,4Y,0.01216,,ATM
4,USD,31.03.2022,Bachelier,1M,5Y,0.01205,,ATM
...,...,...,...,...,...,...,...,...
219,USD,31.03.2022,Bachelier,30Y,10Y,0.00539,,ATM
220,USD,31.03.2022,Bachelier,30Y,15Y,0.00519,,ATM
221,USD,31.03.2022,Bachelier,30Y,20Y,0.00505,,ATM
222,USD,31.03.2022,Bachelier,30Y,25Y,0.00499,,ATM


In [13]:
surfaceATM.to_csv("Swaption_Volas_Normal_USD_ATM.csv", sep=";", decimal=",", index=False)

In [5]:
surface.to_excel("Saption_Volas_LogNormal_USD_31_03_2022.xlsx")